In [ ]:
# Important, set your home directory here:
home_dir = ''

In [ ]:
# Change working directory
import os
os.chdir(home_dir)

from sklearn.metrics import f1_score
from scipy.interpolate import interp1d

import pickle
import numpy as np

import lightgbm as lgb


In [ ]:
%load_ext autoreload
%autoreload 2
from main.models.utils import cv_early_stopping
from main.fairness.paramfitter import BetaEMWD
from main.fairness.wasserstein import WassersteinBinary
from main.fairness.metrics import unfairness

# data loaders
from main.loaders.loader_pubcov import load_coverage_data, prepare_pubcov

In [ ]:
# Set some seeds
for seed_ in [42, 1029, 3948, 103, 56, 93983838, 828, 1928838, 900, 10]:

    data_red = load_coverage_data()

    # Create target variable
    data_red.loc[:, 'PUBCOV'] = np.where(data_red['PUBCOV'] == 1, 1, 0)
    data_red = data_red.assign(low_dummy=np.where(data_red['PINCP'] < 45000, 1, 0))
    data_red = data_red.assign(very_low_dummy=np.where(data_red['PINCP'] < 15000, 1, 0))
    # Drop column with info
    data_red = data_red.drop(columns=['PINCP'])

    X_train, X_test, y_train, y_test, transformer = prepare_pubcov(data_red, 
                                                                    seed=seed_)

    # Keep set of original splits (makes it easier to recover
    # the orignal variable later on)
    X_train_orig = X_train.copy()
    X_test_orig = X_test.copy()

    X_train = np.delete(X_train, 23, axis=1)
    X_test = np.delete(X_test, 23, axis=1)
            
    # Model optimization
    # Fit a basic model
    params = {
        "objective": "binary",
        "metric": "auc",
        "min_data_in_leaf": 50,
        "learning_rate": 0.05,
        "feature_fraction": 0.9,
        "verbose": -1
    }

    cv_results = cv_early_stopping(params=params, 
                                nfolds=3, 
                                max_rounds=1000, 
                                early_stopping_rounds=20, 
                                X_train=X_train, 
                                y_train=y_train, 
                                objective='classification')


    # Fit best model (based on boosting iters)
    best_res = np.argmax(cv_results['metric'])
    best_iter = cv_results['iterations'][best_res]

    # Re-Train on whole dataset
    data_train_all = lgb.Dataset(data=X_train, 
                                label=y_train)

    best_estimator = lgb.train(params=params,
                            train_set=data_train_all, 
                            num_boost_round=best_iter)

    # Run predictions
    preds_uncorrected_calib = best_estimator.predict(X_train)
    preds_uncorrected_test = best_estimator.predict(X_test)

    # Recover full set of sensitive variables
    # and set to sensitive variables
    # Here: 22 and 23 are the variables we created and we assume that 
    # 23 is not observed
    sens_observed_calib = np.where(X_train_orig[:, 22] > 0, 1, 0)
    sens_observed_test = np.where(X_test_orig[:, 22] > 0,1,0) 
    sens_unobserved_test = np.where(X_test_orig[:,23] > 0,1,0)

    # Use a beta model
    preds_uncorrected_calib = best_estimator.predict(X_train)
    preds_uncorrected_test = best_estimator.predict(X_test)

    fairness_trans = WassersteinBinary()
    fairness_trans.fit(preds_uncorrected_calib, sens_observed_calib)

    nonparam_preds_calib = fairness_trans.transform(preds_uncorrected_calib, sens_observed_calib)
    nonparam_preds_test = fairness_trans.transform(preds_uncorrected_test, sens_observed_test)

    parametric_sampler = BetaEMWD()
    parametric_sampler.fit(X=np.array(nonparam_preds_calib), 
                            sampling_obs=len(nonparam_preds_calib)*3)

    parametric_preds = parametric_sampler.sample(n=len(nonparam_preds_test), 
                                                    mc_samples=250)

    interpolator = interp1d(np.sort(nonparam_preds_test),
                            parametric_preds)

    param_preds_test = interpolator(nonparam_preds_test)


    res_dict = {}

    unobs_test = sens_unobserved_test
    sens_test = sens_observed_test

    for eps_ in [0.0, 0.25,0.5,0.75]:
        res_dict[eps_] = {}

        cu_ = 0.21
            
        pred_nonparam_int = eps_*preds_uncorrected_test + (1-eps_)*nonparam_preds_test
        pred_param_int = eps_*preds_uncorrected_test + (1-eps_)*param_preds_test

        
        mse_nonparam = f1_score(y_test, np.where(pred_nonparam_int > cu_, 1, 0))
        mse_param = f1_score(y_test, np.where(pred_param_int > cu_, 1, 0))
        
        unf_np = unfairness(pred_nonparam_int[sens_test == 1], 
                        pred_nonparam_int[sens_test != 1])
        
        unf_p = unfairness(pred_param_int[sens_test == 1], 
                        pred_param_int[sens_test != 1])
        
        unf_np_u = unfairness(pred_nonparam_int[unobs_test == 1], 
                        pred_nonparam_int[unobs_test != 1])
        
        unf_p_u = unfairness(pred_param_int[unobs_test == 1], 
                        pred_param_int[unobs_test != 1])
        
        res_dict[eps_]['qrisk_param'] = mse_param
        res_dict[eps_]['qrisk_nonparam'] = mse_nonparam

        res_dict[eps_]['unfairness_param'] = unf_p
        res_dict[eps_]['unfairness_nonparam'] = unf_np

        res_dict[eps_]['unfairness_unobs_param'] = unf_p_u
        res_dict[eps_]['unfairness_unobs_nonparam'] = unf_np_u

    with open(f'data/results/folktabs/dicts/dict_epsiolon{seed_}.pkl', 'wb') as con_:
        pickle.dump(res_dict, con_)
